In [143]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from data_store import numpy_datastore as npds
from transformer import transformers as trs
from search import inverted_multi_index_searcher as imis
from quantization import pq_quantizer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
productsiftbincount_sqlite_db_path='ds_data\imgid_productsiftbincount'
productsiftbincount_sqlite_table_name='imgid_productsiftbincount'
productsiftbincount_clusters_sqlite_db_path='ds_data\imgid_productsiftbincount_clusters'
productsiftbincount_clusters_sqlite_table_name='clusterid_cluster'

productsiftbincount_ds = sqlds.SQLiteTableDataStore(productsiftbincount_sqlite_db_path, productsiftbincount_sqlite_table_name)
productsiftbincount_clusters_ds = sqlds.SQLiteTableDataStore(productsiftbincount_clusters_sqlite_db_path, productsiftbincount_clusters_sqlite_table_name)

In [145]:
cluster_centers_ndarray_ds=snads.StreamNdarrayAdapterDataStore(productsiftbincount_clusters_ds, detect_final_shape_by_first_elem=True)
cluster_centers=cluster_centers_ndarray_ds.get_items_sorted_by_ids()
print(cluster_centers.shape)

(2, 350, 300)


In [146]:
productsiftbincount_ndarray_ds = snads.StreamNdarrayAdapterDataStore(productsiftbincount_ds, detect_final_shape_by_first_elem=True)
X = productsiftbincount_ndarray_ds.get_items_sorted_by_ids()
X_ids=productsiftbincount_ndarray_ds.get_ids_sorted()
print(X.shape)
print(X_ids.shape)

(999, 600)
(999,)


In [147]:
searcher_=imis.InvertedMultiIndexSearcher(X, X_ids, cluster_centers)

In [148]:
np_neighbor_ids_ds=npds.NumpyDataStore()
steps.search_step(productsiftbincount_ds, searcher_, 6 , np_neighbor_ids_ds)

In [149]:
np_neighbor_ids_ds.get_items_sorted_by_ids()

array([[  1,   2,   8,   7,   5,   4],
       [  2,   3,   9,   6,   8,   7],
       [  3,   2,   9,   6,   8,   7],
       ..., 
       [997, 991, 998, 995, 994, 992],
       [998, 995, 994, 991, 997, 992],
       [999, 996, 993, 992, 998, 995]])